# 1. Data Preparation

This notebook handles two datasets with Trentino's schools data. In particular, two sources have been found: vivoscuola website and aprilascuola project, which offers a hidden API where it is possible to scrape schools details. 

In [32]:
# Setup
import plotly.graph_objects as go
import textwrap
import json
import pandas as pd
import numpy as np

## Vivoscuola dataset
Starting with vivoscuola dataset, the so called "Istituti Comprensivi" are not taken into consideration, since they are a set of educational units, also called schools. This type of institutions are recognizable through the missing field "Scuola".

In [33]:
# VIVOSCUOLA DATASET
vivo = pd.read_csv("../data/vivoscuola.csv", sep=";")

print("Number of rows: "+str(len(vivo)))
vivo.head()

Number of rows: 997


,Istituto Principale,Scuola,Tipo Istituto,Tipo Gestione,Dirigente,Direttore,Coordinatore Pedagogico,Indirizzo,Comune,Telefono,Fax,Email istituto,Email dirigenza,Email segreteria,Sito web,Codice MIUR
0,ASSOCIAZIONE RUDOLF STEINER PER LA PEDAGOGIA A...,Asilo Colle Fiorito,NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Stazione Mori, 10 38068",ROVERETO,0464 434899,NaN,NaN,NaN,NaN,NaN,NaN
1,"Scuola dell'infanzia di Breguzzo - Bondo ""Giu...","Scuola dell'Infanzia di Breguzzo - Bondo ""Giu...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Marconi - Breguzzo, 5 38087",SELLA GIUDICARIE,0465-901550,NaN,breguzzo.bondo.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
2,"Scuola dell'infanzia di Riva ""Giardino""","Scuola dell'Infanzia di Riva ""Giardino""",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Viale Roma, 32 38066",RIVA DEL GARDA,0464-552360,0464-560456,rivagiardino.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
3,"Scuola dell'infanzia di Lavis ""Madre Maddalen...","Scuola dell'Infanzia di Lavis ""Madre Maddalen...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via Dei Colli, 4 38015",LAVIS,0461 240366,0461-249078,lavis.materna@pec.associazionecoesi.com,NaN,NaN,NaN,NaN
4,"Scuola dell'infanzia di Rovereto ""Clementino ...","Scuola dell'Infanzia di Rovereto ""Clementino ...",NaN,Paritaria/equiparata,NaN,NaN,NaN,"Via S Maria, 58 38068",ROVERETO,0464-420061,NaN,asilovannetti@pec.it,NaN,asilovannetti@gmail.com,NaN,NaN


In [34]:
# Remove Istituti Comprensivi
vivo = vivo[~vivo['Scuola'].isnull()]
print("Number of rows: "+str(len(vivo)))

Number of rows: 724


After removing the institutions that include schools, we are left with 724 school units.

Since there is no column that suggests the grade of the school, the following function tries to detect it from the school name itself. Notice that vivoscuola, compared to aprilascuola, contains schools of all grades, starting from kindergarden to professional training. 

In [35]:

# Infer school type by the complete name
def insert_school_type(df, name_index, ist_index):
    type = []
    for i in range(len(df)):
        s = df.iloc[i, name_index].lower()
        t = ""
        if ("asilo" in s) or ("scuola materna" in s) or ("scuola dell'infanzia" in s):
            t = "Scuola dell'Infanzia"
        elif ("scuola primaria" in s) or ("primaria" in s):
            t = "Scuola Primaria"
        elif ("scuola secondaria di primo " in s) or ("secondaria i" in s):
            t = "Scuola Secondaria di Primo Grado"
        elif ("scuola secondaria di secondo " in s) or ("liceo" in s) or ("istituto tecnico" in s) or ("istituto professionale" in s):
            t = "Scuola Secondaria di Secondo Grado"
        elif ("formazione professionale" in s) or ("formazione professionale" in df.iloc[i, ist_index].lower()):
            t = "Formazione professionale"
        elif "educazione per adulti" in s:
            t = "Educazione per adulti"
        else:
            t = np.nan
        type.append(t)
    df['Tipo Istituto'] = type


insert_school_type(vivo, 1, 0)

We can notice the type of school from the column *"Tipo Istituto"*, inspecting some random rows to get more than one single category of schools. 

In [36]:
vivo.sample(5)

,Istituto Principale,Scuola,Tipo Istituto,Tipo Gestione,Dirigente,Direttore,Coordinatore Pedagogico,Indirizzo,Comune,Telefono,Fax,Email istituto,Email dirigenza,Email segreteria,Sito web,Codice MIUR
942,ISTITUTO COMPRENSIVO MORI,"SCUOLA SECONDARIA DI PRIMO GRADO ""B. MALFATTI""...",Scuola Secondaria di Primo Grado,A carattere statale/provinciale,GIOVANNI KRAL,NaN,NaN,"Via Giovanni XXIII, 64 38065",MORI,0464/918669,NaN,ic.mori@pec.provincia.tn.it,dir.ic.mori@scuole.provincia.tn.it,segr.ic.mori@scuole.provincia.tn.it,www.icmori.it/,TNMM84901L
810,ISTITUTO COMPRENSIVO CAVALESE,SCUOLA PRIMARIA MOLINA DI FIEMME,Scuola Primaria,A carattere statale/provinciale,ROBERTO TROLLI,NaN,NaN,"Via Segherie, 64/B 38030",CASTELLO-MOLINA DI FIEMME,0462/340314,0462/342294,ic.cavalese@pec.provincia.tn.it,dir.cavalese@scuole.provincia.tn.it,segr.cavalese@scuole.provincia.tn.it,www.iccavalese.it,TNEE80904A
194,Circolo di coordinamento n. 1,Scuola dell'Infanzia di Casatta,Scuola dell'Infanzia,A carattere statale/provinciale,NaN,NaN,GIULIANA BATTISTI,"Frazione Casatta, 54 38040",VALFLORIANA,0462/910249,0461/493495,scuolainfanzia.casatta@scuole.provincia.tn.it,NaN,circolo.coordinamento01@provincia.tn.it,NaN,NaN
351,ISTITUTO COMPRENSIVO PREDAZZO TESERO PANCHIA' ...,"SCUOLA PRIMARIA ""G. MANEGA"" TESERO",Scuola Primaria,A carattere statale/provinciale,ELISABETTA PIZIO,NaN,NaN,"Via Fia, 11/A 38038",TESERO,0462/813197,0462/500023,ic.predazzo@pec.provincia.tn.it,dir.ic.predazzo@scuole.provincia.tn.it,segr.ic.predazzo@scuole.provincia.tn.it,www.ic-predazzotesero.it,TNEE80804E
875,Scuola dell'infanzia di Tuenno,Scuola dell'Infanzia di Tuenno,Scuola dell'Infanzia,Paritaria/equiparata,NaN,NaN,UTE PANCHER,"Via Marconi - Tuenno, 2 38019",VILLE D'ANAUNIA,0463-451238,NaN,tuenno.presidente@fpsm.tn.it,NaN,tuenno.segretario@fpsm.tn.it,NaN,NaN


Moreover, the column *"Tipo Gestione"* offers a quick view over public (i.e. "Statale") or private schools, also known as charter schools (i.e. "Paritaria"). 

In [37]:
# Map Public and Private school into short version
type = []
for x in vivo['Tipo Gestione']:
    if "Paritaria" in x:
        type.append("Paritaria")
    else:
        type.append("Statale")

vivo['Tipo Gestione'] = type

We can now reorder the columns and remove useless ones. In addition, some columns are renamed, since they will play a mayor role in further code and by being used so frequently, the name should be shorten. 

In [38]:
# Select the columns
vivo = vivo[['Istituto Principale', 'Scuola', 'Tipo Istituto',
             'Tipo Gestione', 'Indirizzo', 'Comune', 'Telefono',
             'Fax', 'Email istituto', 'Email segreteria', 'Sito web', 'Codice MIUR']]

vivo.rename(columns={
    'Istituto Principale': 'Istituto',
    'Scuola': 'Nome',
    'Tipo Gestione': 'Gestione'
}, inplace=True)

The following chunk is necessary to extract the code of the municipality (i.e. CAP) and the address without this code, such that geocoding in the next notebook will be easier. 

In [39]:
# Separate CAP and Address
vivo['CAP'] = [x[-5:] for x in vivo['Indirizzo']]
vivo['Indirizzo'] = [x[:-6] for x in vivo['Indirizzo']]

We can save this dataset inside the Trentino directory inside data, so that original data is not replaced. 

In [40]:
# Save vivo dataset
vivo.reset_index(drop=True, inplace=True) # Resetting properly the index from 0 to n-1
vivo.to_csv("../data/trentino/vivoscuole.csv", index=False)

## Aprilascuola dataset

Aside from vivoscuola dataset, there's also aprilascuola, a project born in the Province of Trento to share school information with citizens. This dataset includes school coordinates and the provincial code, useful to request the number of students and classes per school and per grade. However, it does not consider kindergardens, which instead are present inside vivoscuola dataset. 

Aprilascuola dataset is saved in JSON format and then imported as dataframe. Since the function `pd.read_json()` reads every number as numeric, the school code needs to be assigned separately as strings in order to keep the initial zeros. Also, most of columns are renamed. 

In [42]:
# APRILASCUOLA DATASET (no kindergarden)
apri = pd.read_json(
    "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/ricerca", 
    dtype={'codiceProvinciale':'str', 'idPadre':'int'})
# Rename columns
apri.rename(columns={
    'idPadre': 'Id Istituto',
    'codiceProvinciale': 'Id',
    'codiceMiur': 'Codice MIUR',
    'denominazioneUfficiale': 'Nome',
    'latitudeY': 'lat',
    'longitudeX': 'lon',
    'istituzionePadre': 'Istituto',
    'indirizzo': 'Indirizzo',
    'email': 'Email',
    'telefono': 'Telefono',
    'comune': 'Comune'
}, inplace=True)
apri.drop(['idobj'], axis=1, inplace=True)
apri.head()

,Id Istituto,Id,Codice MIUR,Nome,Indirizzo,Email,Telefono,lat,lon,Istituto,Comune
0,NaN,130,None,"Scuola dell'infanzia di Tione di Trento ""Don G...",Piazza Battisti 4,None,0465 321314,46.034436,10.726353,"{'idobj': None, 'idPadre': None, 'codiceProvin...",TIONE DI TRENTO
1,NaN,0222055913,TNPS11000Q,"LICEO LINGUISTICO ""S. M. Scholl"" - TRENTO",Via Mattioli 8,segr.linguisticotrento@scuole.provincia.tn.it,0461914499,46.059356,11.121925,"{'idobj': None, 'idPadre': None, 'codiceProvin...",TRENTO
2,2767549.0,0222057135,TNPS11000Q,LICEO LINGUISTICO,Via Mattioli 8,None,None,46.059356,11.121925,"{'idobj': 2767549, 'idPadre': None, 'codicePro...",TRENTO
3,NaN,0222059503,TNTD18000T,"ISTITUTO TECNICO ECONOMICO ""A. Tambosi"" - TRENTO",Via Brigata Acqui 19,segr.tambositn@scuole.provincia.tn.it,0461239955,46.065537,11.129403,"{'idobj': None, 'idPadre': None, 'codiceProvin...",TRENTO
4,2767532.0,0222055406,TNTD180507,ISTITUTO TECNICO PER IL SETTORE ECONOMICO SERALE,Via Brigata Acqui 19,None,None,46.065537,11.129403,"{'idobj': 2767532, 'idPadre': None, 'codicePro...",TRENTO


The column about *"Istituti Comprensivi"* is called *Istituto* and contains nested information about the main institute that encloses individual schools. The following chunk will extract their provincial code and their name. 

In [44]:
# Getting nested information outside
apri['Id Istituto'] = [x['codiceProvinciale'] for x in apri['Istituto']]
apri['Istituto'] = [x['denominazioneUfficiale'] for x in apri['Istituto']]

Since most of the string columns are in capslock, these columns, both from vivo and aprilascuola dataset, are transformed such that only the first letters of each word will be in uppercase. 

In [45]:
# Remove capslock and convert everything to Title
apri[['Istituto', 'Nome', 'Comune']] = apri[['Istituto', 'Nome', 'Comune']
                                            ].applymap(lambda s: s.title() if s != None else None)
vivo[['Istituto', 'Nome', 'Comune']] = vivo[['Istituto', 'Nome', 'Comune']
                                            ].applymap(lambda s: s.title() if s != None else None)

A curious thing to be noticed is that aprilascuola schools should all be included inside the vivoscuola dataset. This happens for all schools except for those whose names is *"Educazione Libera Per Adulti"*. By inspecting the JSON page of these schools, we may notice that they lead to the same information of training education (i.e. *"Formazione Professionale"*), which are included inside *Educazione Libera Per Adulti* (treated as their main institute). This means that these schools data were counted twice. For this reason, they are excluded from aprilascuola dataset.

Moreover, "Istituti Comprensivi" provincial codes are removed. In the end, the following chunk inserts the type of school based on columns 3 and 9 (i.e. school name and institute name).

In [46]:
# Remove Institutes and Remove "Educazione Libera Per Adulti", since these schools are counted twice
apri = apri[(~apri['Id Istituto'].isna()) & (
    apri['Nome'] != "Educazione Libera Per Adulti")]

insert_school_type(apri, 3, 9)

## Mixing data sources

Now that both datasets are cleaned, we can join them, keeping attention to possible repetitions. First, we can distinguish schools between kindergardens (i.e. *"materne"*) and non kindergardens (i.e. *"non_mat"*). The last category represents common schools between apri and vivo, which are joined based on the school's name and affiliated institute. 

In [47]:
# Getting the list of kindergarden and not schools
materne = vivo[vivo['Tipo Istituto'] == "Scuola dell'Infanzia"]
non_mat = vivo[vivo['Tipo Istituto'] != "Scuola dell'Infanzia"]

len(materne)
len(non_mat)

# Merging common schools inside vivo and apri
common = pd.merge(apri, non_mat, how="inner", on=['Nome', 'Istituto'])

In [15]:
common.head()

,Id Istituto,Id,Codice MIUR_x,Nome,Indirizzo_x,Email,Telefono_x,lat,lon,Istituto,...,Gestione,Indirizzo_y,Comune_y,Telefono_y,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR_y,CAP
0,0222055913,0222057135,TNPS11000Q,Liceo Linguistico,Via Mattioli 8,None,None,46.059356,11.121925,"Liceo Linguistico ""S. M. Scholl"" - Trento",...,Statale,"Via Mattioli, 8",Trento,0461914499,0461915472,linguisticotrento@pec.provincia.tn.it,segr.linguisticotrento@scuole.provincia.tn.it,www.linguisticotrento.it,TNPS11000Q,38122
1,0222059503,0222055406,TNTD180507,Istituto Tecnico Per Il Settore Economico Serale,Via Brigata Acqui 19,None,None,46.065537,11.129403,"Istituto Tecnico Economico ""A. Tambosi"" - Trento",...,Statale,"Via Brigata Acqui, 19",Trento,0461239955,0461230175,tambosibattisti@pec.provincia.tn.it,segr.tambositn@scuole.provincia.tn.it,http://tambosi.tn.it,TNTD180507,38122
2,0222059503,0222055431,TNTD18000T,Istituto Tecnico Per Il Settore Economico,Via Brigata Acqui 19,None,None,46.065537,11.129403,"Istituto Tecnico Economico ""A. Tambosi"" - Trento",...,Statale,"Via Brigata Acqui, 19",Trento,0461239955,0461230175,tambosibattisti@pec.provincia.tn.it,segr.tambositn@scuole.provincia.tn.it,http://tambosi.tn.it,TNTD18000T,38122
3,0222222901,0222223001,TNMM846015,"Scuola Secondaria Di Primo Grado ""A. Frank"" Vi...",Via Stockstadt Am Rhein 3,None,None,45.918206,11.031039,Istituto Comprensivo Villa Lagarina,...,Statale,"Via Stockstadt Am Rhein, 3",Villa Lagarina,0464/411312,0464/411705,ic.villalagarina@pec.provincia.tn.it,segr.ic.villalagarina@scuole.provincia.tn.it,www.icvillalagarina.it,TNMM846015,38060
4,0222222901,0221442101,TNEE846027,"Scuola Primaria ""R. Galvagni"" Pomarolo",Piazzale Colonello Angheben 5,None,None,45.928203,11.042493,Istituto Comprensivo Villa Lagarina,...,Statale,"Piazzale Colonello Angheben, 5",Pomarolo,0464/411312,0464/411705,ic.villalagarina@pec.provincia.tn.it,segr.ic.villalagarina@scuole.provincia.tn.it,www.icvillalagarina.it,TNEE846027,38060


Let's check whether all Ids inside apri dataset are inside the common dataset created from inner join:

In [50]:
# Check if all apri rows are inside the joined dataframe
for i in apri['Id'].index:
    if apri['Id'][i] not in list(common['Id']):
        print(i)

Since no index is printed, all schools inside apri dataset are inside common dataset. 
Considering vivo non-kindergartens schools, some of them are not included inside common. We can check these names with a right join between apri and non_mat datasets based on name and institute of schools. Whenever the Id is missing, the school is present in vivo but not in apri and we can therefore focus on the missing schools of common dataset. 

In [63]:
non_mat.reset_index(inplace=True)

In [64]:
missing_vivo = pd.merge(apri, non_mat, how="right", on=['Nome', 'Istituto'])
missing_vivo = missing_vivo[missing_vivo['Id'].isna()]

As can be noticed, there are 8 schools that are not inside common dataset, whose information is only detained inside vivo dataset:

In [73]:
non_mat.set_index('index',inplace=True)

Now we can concatenate the inner join dataset with the right join on non_mat dataset to get all the not kindergarten schools, but first let's rename and drop some columns. Notice that since the only rows where MIUR code does not coincide both on x and y version are those with missing values: 

In [57]:
common[common['Codice MIUR_x'] != common['Codice MIUR_y']]

,Id Istituto,Id,Codice MIUR_x,Nome,Indirizzo_x,Email,Telefono_x,lat,lon,Istituto,...,Gestione,Indirizzo_y,Comune_y,Telefono_y,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR_y,CAP
14,0221049598,2013,None,Settore Servizi Serale,Via Ziehl 5,None,None,NaN,NaN,Istituto Formazione Professionale Alberghiero ...,...,Statale,"Via Ziehl, 5",Levico Terme,0461/706294,0461/702025,ifpa.levicoterme@pec.provincia.tn.it,segr.ifpa.levicoterme@scuole.provincia.tn.it,www.alberghierolevico.it,NaN,38056
270,0222059599,0222059694,None,Settore Servizi Serale,Via Borsieri 2,None,None,NaN,NaN,Centro Formazione Professionale Upt - Trento,...,Paritaria,"Via Borsieri, 2",Trento,0461/239997,0461/260235,cfp-upt@pec.it,segreteria.sede@cfp-upt.it,www.cfp-upt.it,NaN,38122
274,0221619599,0221619695,None,Settore Industria E Artigianato Serale,Piazzale Orsi 1,None,None,45.891881,11.034906,Giuseppe Veronesi - Centro Di Istruzione Scola...,...,Paritaria,"Piazzale Orsi, 1",Rovereto,0464/433484,0464/436873,iscrizioni.veronesi@pec.it,segreteria.dir@cfpgveronesi.it,www.cfpgveronesi.it,NaN,38068
311,0222059597,0222059692,None,Settore Industria E Artigianato Serale,Via Asiago 14,None,None,46.045498,11.137545,Centro Formazione Professionale Enaip - Villaz...,...,Paritaria,"Via Asiago, 14",Trento,0461/920386,0461/914935,cfp.villazzano@pec.it,cfp.villazzano@enaip.tn.it,www.enaiptrentino.it,NaN,38123
320,0221619597,0221619685,None,Settore Servizi Serale,Viale Dei Colli 17,None,None,45.890608,11.047722,Istituto Formazione Professionale Alberghiero ...,...,Statale,"Viale Dei Colli, 17",Rovereto,0464/439164,0464/435851,ifpa.rovereto@pec.provincia.tn.it,segr.ifpa.rovereto@scuole.provincia.tn.it,www.alberghierorovereto.it,NaN,38068
322,0222059595,0222059670,None,Settore Servizi Serale,Viale Verona 141,None,None,46.052671,11.131164,Istituto Formazione Professionale Servizi Alla...,...,Statale,"Viale Verona, 141",Trento,0461/933147,0461/931682,ifp.trento@pec.provincia.tn.it,ifp.trento@scuole.provincia.tn.it,https://ifpsandropertinitrento.it,NaN,38123


Since vivoscuola addresses and contact information are well formatted and more complete than aprilascuola's, we will keep them and discard their "y" version belonging to apri dataset:

In [58]:
# Keep vivoscuola addresses and contact information, since they are well formatted and complete
common.rename(columns={
    'Indirizzo_y': 'Indirizzo',
    'Telefono_y': 'Telefono',
    'Comune_y': 'Comune',
    'Tipo Istituto_y': 'Tipo Istituto',
    'Codice MIUR_y': 'Codice MIUR'}, inplace=True)
common.drop(['Codice MIUR_x', 'Indirizzo_x', 'Telefono_x', 'Email',
             'Comune_x', 'Tipo Istituto_x', ], axis=1, inplace=True)

Now we can concatenate common dataset with not-kindergarden's whose name, address and institute match with those inside missing_vivo dataset.

In [76]:
# 2. Add the remaining missing schools in Vivo
common = pd.concat([common, non_mat.loc[list(missing_vivo['index'])]], axis=0, ignore_index=True)

By re-executing the code forehead mentioned, we can recheck whether all non kindergarden schools inside vivo are present in common dataset. Since the following chunk does not print anything, we can state that all schools are inside common:

In [77]:
# Check if all vivo rows are inside the joined dataframe
indexes = []
for i in non_mat['Nome'].index:
    if non_mat['Nome'][i] not in list(common['Nome']):
        indexes.append(i)
        print(i)

We have collected information about primary, middle and high schools, plus training education schools. We can concatenate these schools with kindergardens saved inside materne dataset to get the complete dataset of all schools in Trentino. 

In [80]:
# Adding scuole materne to the dataset, inserting None where information is missing
common = pd.concat([common, materne], axis=0, ignore_index=True)

Navigating across vivoscuola, an error can be noticed for a **primary school in Cavedine**. It seems like there are two of them (the other is called *Scuola Primaria Vigo Cavedine*), with same data. Browsing inside the school's website, we can notice just one primary school in Cavedine. Since most updated information can be retrieved with ["Scuola Primaria Cavedine"](https://www.vivoscuola.it/Scuole/ISTITUTO-COMPRENSIVO-VALLE-DEI-LAGHI-DRO2/SCUOLA-PRIMARIA-CAVEDINE/(offset)/scuola/(data)/alunniclassi), the other one is erased.

In [81]:
common.loc[(common['Nome'].str.contains("Cavedine")) & (common['Tipo Istituto']=="Scuola Primaria")]

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP
444,0222152901,0220532101,Scuola Primaria Cavedine,45.978474,10.976432,Istituto Comprensivo Valle Dei Laghi - Dro,Scuola Primaria,Statale,"Via Don Negri, 21",Cavedine,0461/864026,0461/340648,ic.valledeilaghi@pec.provincia.tn.it,segr.ic.valledeilaghi@scuole.provincia.tn.it,www.icvalledeilaghidro.it,TNEE84406Q,38073
456,NaN,NaN,Scuola Primaria Vigo Cavedine,NaN,NaN,Istituto Comprensivo Valle Dei Laghi - Dro,Scuola Primaria,Statale,"Via Don Negri, 21",Cavedine,0461/864026,0461/340648,ic.valledeilaghi@pec.provincia.tn.it,segr.ic.valledeilaghi@scuole.provincia.tn.it,www.icvalledeilaghidro.it,TNEE84407R,38073


In [82]:
# Correcting information about a school
common.drop(common.loc[(common['Nome'] == "Scuola Primaria Vigo Cavedine") 
                       & (common['Id'].isna())].index, 
            inplace=True)

Let's check if there are some other duplicates, based on name, institute, municipality and address:

In [83]:
# Check if there are some duplicates
common[common.duplicated(
    subset=['Nome', 'Istituto', 'Comune', 'Indirizzo'], keep=False)]

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP


In the end, the dataset can be saved. The pickle format was chosen in order to preserve data types, in particular related to the Provincial Code of the school that may be necessary to retrieve additional information in aprilascuola API. 

In [90]:
common.reset_index(drop=True, inplace=True)

In [85]:
common.to_pickle("../data/Trentino/scuole.pkl")

In [86]:
# Importing the file
schools = pd.read_pickle("../data/Trentino/scuole.pkl")

## Visualizations

In this short section, we will explore the distribution of schools between public and private, but also in terms of type of school (i.e. primary, kindergarden, middle, high and training schools). Since the plot is used within the website, it is interactive and built with plotly. Two main actions can be performed:

1. Hover: when hovering with the cursor, it is possible to read a description, the count (number of schools of that category) and the percentage with respect to the parent category. In the case of public and private school the parent is the total number of schools in Trentino; whereas, related to the specific type of school, the parent is the type of management of the school (i.e. public or private);
2. Click: When clicking on the specific category, the sunburst will be focused on that specific category. 

In [87]:
# Grouping by private/public and type of school
df = schools.groupby(['Gestione', 'Tipo Istituto']
                     ).size().to_frame('count').reset_index()
df.loc[len(df)] = ['', 'Paritaria', sum(
    df[df['Gestione'] == 'Paritaria']['count'])]
df.loc[len(df)] = ['', 'Statale', sum(
    df[df['Gestione'] == 'Statale']['count'])]
df.index = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 0, 6]
df = df.sort_index()
df['ids'] = df['Gestione']+df['Tipo Istituto']
df['colors'] = ["#DD5840","#F16D55","#F16D55","#F16D55","#F16D55","#F16D55",
                "#5A7FAF", "#7594BD","#7594BD","#7594BD","#7594BD","#7594BD","#7594BD"]


def customwrap(s, width=13):
    return "<br>".join(textwrap.wrap(s, width=width))


df['Tipo Istituto'] = df['Tipo Istituto'].map(customwrap)

In [88]:
# Show proportion of schools per type and management
import plotly.graph_objects as go
fig = go.Figure(go.Sunburst(values=df["count"],
                            ids=df['ids'],
                            labels=df["Tipo Istituto"],
                            parents=df["Gestione"],
                            branchvalues='total',
                            hoverinfo="label+value+percent parent",
                            insidetextorientation='radial',
                            marker=dict(colors=df.colors),
                            maxdepth=2))
fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))
fig.show()
fig.write_html("../viz/sunburst_schools.html")